In [ ]:
import sys
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
from himpy.histogram import operations, expressionOperations
from himpy.executor import Parser, Evaluator
from himpy.utils import E

In [ ]:
%load_ext autoreload
%autoreload 2

# dataset generator
from utils.datasets import ColorImageGenerator

# feature extraction
from utils.feature_extraction import (
    FeatureMerger,
    ColorSetTransformer,
    create_histogram,
    create_histogram_
)

# search engine
from utils.search_engine import SearchEngine

# plot
from utils.plot.plotly_plot import (
    show_color_elements,
    show_complete_histogram,
    show_rank_images
)

# image colors
from utils.feature_extraction.color import COLOR_ELEMENTS, COLOR_ELEMENTS_RGB

In [ ]:
image_generator = ColorImageGenerator()
color_transformer = ColorSetTransformer()
parser = Parser()

In [ ]:
# Definition of high-level positional elements

Ec_green        = E("e1+e2+e3+e4+e5+e6+e7+e8+e9+e10+e11+e12+e13+e14+e15+e16+e17+e18+e19+e20")
Ec_yellow_green = E("e2+e3+e21+e22+e23+e24+e25+e26+e27+e28+e29+e30")
Ec_red          = E("e31+e32+e33+e34+e35+e36+e37+e38+e39+e40")
Ec_rose         = E("e32+e35+e36+e39+e40")


Ecs = [
    ("green", Ec_green),
    ("yellow_green", Ec_yellow_green),
    ("red", Ec_red),
    ("rose", Ec_rose)
]


# Sets of hight-level color elements (they will be used for the Evaluator below)
Ecs_set = { name: parser.parse_set(Ec.value) for name, Ec in Ecs}
high_level_elements = Ecs_set

## Image Retrieval

In [ ]:
# Images with normal distrubited some elements 
images = [
    image_generator.generate(
        shape=(100, 100), 
        steps=(20, 20)) 
    for i in range(1000)
]

In [ ]:
# Create histograms for the images
hists = list()
limit = len(images)

for indx, image in enumerate(images):
    color_image = color_transformer.fit_transform(X=image, y=None)
    hist = create_histogram((color_image,))
    hists.append((indx, hist))
    print("\rCurrent image index: {}/{}".format(indx + 1, limit), end="")

In [ ]:
import pickle

with open("hists.pkcl", "wb") as f:
    pickle.dump(hists, f)
with open("images.pkcl", "wb") as f:
    pickle.dump(images, f)

In [ ]:
# Initialize a search engine
evaluator = Evaluator(operations, expressionOperations, high_level_elements=high_level_elements)
search_engine_default = SearchEngine(hists, parser, evaluator, use_index=False)
search_engine_inverted_index = SearchEngine(hists, parser, evaluator, use_index=True)

In [ ]:
TOP_N = 20

In [ ]:
# Elements
E1 = E("green")
E2 = E("yellow_green")
E3 = E("red")
E4 = E("rose")
E5 = E("any")

# Define your query
query = E3 - E4

# Retrieve images using the query
ranked_images = search_engine_inverted_index.retrieve(query, topN=TOP_N)
print("Total retrieved images:", len(ranked_images))
ranked_images

In [ ]:
fig = show_rank_images(images, ranked_images, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()

In [ ]:
for id, score in ranked_images:
    green = evaluator.eval(["green"], hists[id][1]).sum() * 25
    yellow_green = evaluator.eval(["yellow_green"], hists[id][1]).sum() * 25
    red = evaluator.eval(["red"], hists[id][1]).sum() * 25
    rose = evaluator.eval(["rose"], hists[id][1]).sum() * 25
    print(f"id: {id:>2}, green: {round(green):>2}, yellow: {round(yellow_green):>2}, red: {round(red):>2}, rose: {round(rose):>2}")

In [ ]:
ranked_images2 = search_engine_default.retrieve(query, topN=TOP_N)
print("Total retrieved images:", len(ranked_images2))
ranked_images2[:5]

In [ ]:
fig = show_rank_images(images, ranked_images2, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()